# Forward Simulation

In this notebook, we compute a single line of AEM data over a conductive plate in a resistive background. We plot the currents and magnetic fields in the subsurface. 

This notebook generates Figures 1-4 in: 

Heagy, L., Kang, S., Cockett, R., and Oldenburg, D., 2018, _Open source software for simulations and inversions of airborne electromagnetic data_, AEM 2018 International Workshop on Airborne Electromagnetics 

In [ ]:
from SimPEG import EM, Mesh, Maps, Utils
import numpy as np
from scipy.constants import mu_0
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import animation, collections
from pymatsolver import Pardiso

import ipywidgets 
from IPython.display import HTML

%matplotlib inline

In [ ]:
from matplotlib import rcParams
rcParams['font.size'] = 16

## Create a Tensor Mesh

Here, we build a 3D Tensor mesh to run the forward simulation on. 

In [ ]:
cs, ncx, ncy, ncz, npad = 50., 20, 1, 20, 10
pad_rate = 1.3
hx = [(cs,npad,-pad_rate), (cs,ncx), (cs,npad,pad_rate)]
hy = [(cs,npad,-pad_rate), (cs,ncy), (cs,npad,pad_rate)]
hz = [(cs,npad,-pad_rate), (cs,ncz), (cs,npad,pad_rate)]

mesh = Mesh.TensorMesh([hx,hy,hz], 'CCC')

In [ ]:
print(mesh)

In [ ]:
# print diffusion distance and make sure mesh padding goes beyond that
print("diffusion distance: {:1.2e} m".format(1207*np.sqrt(1e3*2.5*1e-3)))
print("mesh extent : {:1.2e} m".format(mesh.hx[-10:].sum()))

## Build a model

In [ ]:
# model parameters
sig_air = 1e-8
sig_half = 1e-3
sig_plate = 1e-1

In [ ]:
# put the omdel on the mesh 
blk1 = Utils.ModelBuilder.getIndicesBlock(
    np.r_[-49.5, 500, -50],
    np.r_[50, -500, -450], 
    mesh.gridCC
)

sigma = np.ones(mesh.nC)*sig_air
sigma[mesh.gridCC[:,2]<0.] = sig_half
sigma_half = sigma.copy()
sigma[blk1] = sig_plate

In [ ]:
xref = 0
zref = -100.
yref = 0.

xlim = [-700., 700.]
ylim = [-700., 700.]
zlim = [-700., 0.]

indx = int(np.argmin(abs(mesh.vectorCCx-xref)))
indy = int(np.argmin(abs(mesh.vectorCCy-yref)))
indz = int(np.argmin(abs(mesh.vectorCCz-zref)))

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (12,6))

clim = [1e-3, 1e-1]

dat1 = mesh.plotSlice(sigma, grid=True, ax=ax[0], ind=indz, pcolorOpts={'norm':LogNorm()}, clim=clim)
dat2 = mesh.plotSlice(sigma, grid=True, ax=ax[1], ind=indx, normal='X', pcolorOpts={'norm':LogNorm()}, clim=clim)

cb = plt.colorbar(dat2[0], orientation="horizontal", ax = ax[1])

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[1].set_xlim(xlim)
ax[1].set_ylim(ylim[0], 0.)

plt.tight_layout()

ax[0].set_aspect(1)
ax[1].set_aspect(1)

ax[0].set_title("Conductivity at Z={:1.0f}m".format(mesh.vectorCCz[indz]))
ax[1].set_title("X={:1.0f}m".format(mesh.vectorCCx[indx]))

cb.set_label("$\sigma$ (S/m)")

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (12, 6))

clim = [1e-3, 1e-1]

dat1 = mesh.plotSlice(sigma, grid=True, ax=ax[0], ind=indz, pcolorOpts={'norm':LogNorm()}, clim=clim)
dat2 = mesh.plotSlice(sigma, grid=True, ax=ax[1], ind=indy, normal='Y', pcolorOpts={'norm':LogNorm()}, clim=clim)

cb = plt.colorbar(dat2[0], orientation="horizontal", ax = ax[1])

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[1].set_xlim(xlim)
ax[1].set_ylim(zlim)

plt.tight_layout()

ax[0].set_aspect(1)
ax[1].set_aspect(1)

ax[0].set_title("Conductivity at Z={:1.0f} m".format(mesh.vectorCCz[indz]))
ax[1].set_title("X= {:1.0f}m".format(mesh.vectorCCx[indx]))

cb.set_label("$\sigma$ (S/m)")

## Assemble the survey

In [ ]:
x = mesh.vectorCCx[np.logical_and(mesh.vectorCCx>-450, mesh.vectorCCx<450)]
# np.save("x", x)

In [ ]:
%%time

# assemble the sources and receivers
time = np.logspace(np.log10(5e-5), np.log10(2.5e-3), 21)
srcList = []

for xloc in x:
    location = np.array([[xloc, 0., 30.]])
    rx_z = EM.TDEM.Rx.Point_dbdt(location, time, 'z')
    rx_x = EM.TDEM.Rx.Point_dbdt(location, time, 'x')
    src = EM.TDEM.Src.CircularLoop([rx_z, rx_x], orientation='z', loc=location)
    srcList.append(src)

## Set up the forward simulation

In [ ]:
timesteps = [(1e-05, 15), (5e-5, 10), (2e-4, 10)]

In [ ]:
prb = EM.TDEM.Problem3D_b(mesh, Solver=Pardiso, verbose=True, timeSteps=timesteps, sigmaMap=Maps.IdentityMap(mesh))
survey = EM.TDEM.Survey(srcList)
survey.pair(prb)

## Solve the forward simulation

In [ ]:
%%time
fields = prb.fields(sigma)

## Compute predicted data

In [ ]:
dpred = survey.dpred(sigma, f=fields)
DPRED = dpred.reshape((survey.nSrc, 2, rx_z.times.size))

# uncomment to add noise
# noise = abs(dpred)*0.05 * np.random.randn(dpred.size) + 1e-14
# dobs = dpred + noise
# DOBS = dobs.reshape((survey.nSrc, 2, rx_z.times.size))

In [ ]:
# # uncomment to save
# np.save("dobs", dobs)
# np.save("dpred", dpred)

In [ ]:
# 0, 1e-4, 4e-4
rx_z.times[[0, 4, 12]]

In [ ]:
fig = plt.figure(figsize=(9, 3.5), dpi=350)
for itime in range(rx_z.times.size):
    plt.semilogy(x, -DPRED[:,0,itime], 'k.-', lw=1)
# plt.plot(srcList[5].loc[0][0]*np.ones(3), -DPRED[5,0,[0, 4, 12]] , 's', color="C3")
plt.xlabel("x (m)")
plt.ylabel("Voltage (V/Am$^2$)")
plt.grid(which="both", alpha=0.2)

# Plot the fields

In [ ]:
def plot_currents(itime, iSrc, clim=None, ax=None, showcb=True, showit=True):
    if ax is None: 
        fig, ax = plt.subplots(1,2, figsize = (16,8))
    
    location  = srcList[iSrc].loc
    S = np.kron(np.ones(3), sigma)
    j = Utils.sdiag(S) * mesh.aveE2CCV * fields[srcList[iSrc], 'e', itime]
    
    dat1 = mesh.plotSlice(
        j, normal='Z', ind=int(indz), vType='CCv', view='vec', ax=ax[0], 
        range_x=xlim, range_y=ylim, sample_grid = [cs, cs],
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = 1e-12 if clim is not None else None
    )
    dat2 = mesh.plotSlice(
        j, normal='X', ind=int(indx), vType='CCv', view='vec', ax=ax[1], 
        range_x=xlim, range_y=zlim,
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'}, 
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = 1e-12 if clim is not None else None
    )
    ax[0].plot(location[0], location[1], 'go', ms=20)
    ax[0].text(-600, 500, "Time: {:1.2f} ms".format(prb.times[itime]*1e3), color='w', fontsize=20)
    
    if showcb is True: 
        cb = plt.colorbar(dat1[0], orientation="horizontal", ax = ax[1])
        cb.set_label("Current density (A/m$^2$)")
    
    ax[0].set_aspect(1)
    ax[1].set_aspect(1)
    ax[0].set_title("Current density at Z={:1.0f}m".format(mesh.vectorCCz[indz]))
    ax[1].set_title("X={:1.0f}m".format(mesh.vectorCCx[indx]))
    
    
    
    if showit: 
        plt.show()
    
    return [d for d in dat1 + dat2]

## View the current density through time

In [ ]:
ipywidgets.interact(
    plot_currents,
    itime = ipywidgets.IntSlider(min=1, max=len(prb.times), value=1),
    iSrc = ipywidgets.IntSlider(min=0, max=len(srcList), value=5),
    clim = ipywidgets.fixed([3e-13, 2e-9]),
    ax=ipywidgets.fixed(None), 
    showit=ipywidgets.fixed(True),
    showcb=ipywidgets.fixed(True)
)


In [ ]:
def plot_magnetic_flux(itime, iSrc, clim=None, ax=None, showcb=True, showit=True):
    if ax is None: 
        fig, ax = plt.subplots(1,1, figsize = (8,8))
    
    location  = srcList[iSrc].loc
    b = mesh.aveF2CCV * fields[survey.srcList[iSrc], 'b', itime]
    
    dat1 = mesh.plotSlice(
        b, normal='Y', ind=int(indy), vType='CCv', view='vec', 
        range_x=xlim, range_y=xlim,
        ax=ax, pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = clim[0] if clim is not None else None
    )

    ax.plot(location[0], location[2], 'go', ms=15)
    
    ax.text(-600, 500, "Time: {:1.2f} ms".format(prb.times[itime]*1e3), color='w', fontsize=20)
    
    if showcb: 
        cb = plt.colorbar(dat1[0], ax = ax)
        cb.set_label("Magnetic Flux Density (T)")
    ax.set_aspect(1)
    ax.set_title("Magnetic Flux Density at Y={:1.0f}m".format(mesh.vectorCCy[indy]))
    
    if showit: 
        plt.show()
    
    return [d for d in dat1 + dat2]

In [ ]:
ipywidgets.interact(
    plot_magnetic_flux,
    itime = ipywidgets.IntSlider(min=1, max=len(prb.times), value=1),
    iSrc = ipywidgets.IntSlider(min=0, max=len(srcList), value=5),
    clim = ipywidgets.fixed([1e-17, 3e-14]),
    ax=ipywidgets.fixed(None), 
    showit=ipywidgets.fixed(True),
    showcb=ipywidgets.fixed(True)
)


## Print Figures

In [ ]:
fig, axes = plt.subplots(3,2, figsize = (12,6*3))
 
iSrc = 5
clim = [3e-13, 2e-9]

for i, itime in enumerate([1, 10, 22]):
    
    ax = axes[i, :]
    location  = srcList[iSrc].loc
    
    S = np.kron(np.ones(3), sigma)
    j = Utils.sdiag(S) * mesh.aveE2CCV * fields[srcList[iSrc], 'e', itime]

    dat1 = mesh.plotSlice(
        j, normal='Z', ind=int(indz), vType='CCv', view='vec', ax=ax[0], 
        range_x=xlim, range_y=ylim, 
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = 1e-12 if clim is not None else None
    )
    dat2 = mesh.plotSlice(
        j, normal='X', ind=int(indx), vType='CCv', view='vec', ax=ax[1], 
        range_x=xlim, range_y=zlim,
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'}, 
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = 1e-12 if clim is not None else None
    )
    
    ax[0].plot(location[0,0], location[0,1], 'go', ms=20)
    ax[0].text(-600, 500, "Time: {:1.2f} ms".format(prb.times[itime]*1e3), color='w', fontsize=20)
    cb = plt.colorbar(dat1[0], orientation="horizontal", ax = ax[1])
    cb.set_label("Current density (A/m$^2$)")
    ax[0].set_aspect(1)
    ax[1].set_aspect(1)
    
    
    if itime == 1: 
        ax[0].set_title("Current density at Z={:1.0f}m".format(mesh.vectorCCz[indz]))
        ax[1].set_title("X={:1.0f}m".format(mesh.vectorCCx[indx]))
    else:
        ax[0].set_title("")
        ax[1].set_title("")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (15, 5))

fig.subplots_adjust(bottom=0.7)

iSrc = 5
clim = [1e-17, 3e-14]

for a, itime, title in zip(ax, [1, 10, 22], ["a", "b", "c"]):
    location  = srcList[iSrc].loc
    
    b = mesh.aveF2CCV * fields[survey.srcList[iSrc], 'b', itime]
    
    dat1 = mesh.plotSlice(
        b, normal='Y', ind=int(indy), vType='CCv', view='vec', 
        range_x=xlim, range_y=xlim,
        ax=a, pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim, stream_threshold = clim[0] if clim is not None else None
    )
        
    if itime > 1:
        a.get_yaxis().set_visible(False)

    a.plot(location[0], location[2], 'go', ms=15)
    
    a.text(-600, 500, "Time: {:1.2f} ms".format(prb.times[itime]*1e3), color='w', fontsize=20)
    a.set_aspect(1)
    a.set_title("({})".format(title))
#     a.set_title(("Magnetic Flux Density at Y=%.0fm")%(mesh_core.vectorCCy[indy]))
    

plt.tight_layout()


cbar_ax = fig.add_axes([0.2, -0.05, 0.6, 0.05])
cb = fig.colorbar(dat1[0], cbar_ax, orientation='horizontal')
cb.set_label('Magnetic Flux Density (T)')

## Movie of the currents and the magnetic flux

Movies shown in the presentation. It is slow to build these, so by default we won't. Change `make_movie` to `True` to build the movies

In [ ]:
make_movie = True
save_mp4 = True

In [ ]:
iSrc = 5

if make_movie: 

    fig, ax = plt.subplots(1, 2, figsize = (16,8), dpi=290)
    out = plot_currents(itime=1, iSrc=iSrc, clim=[3e-13, 2e-9], ax=ax, showcb=True, showit=False)

    def init():
        [o.set_array(None) for o in out if isinstance(o, collections.QuadMesh)]
        return out

    def update(t):
        for a in ax:
            a.patches = []
            a.lines = []
            a.clear()
        return plot_currents(itime=t, iSrc=iSrc, clim=[3e-13, 2e-9], ax=ax, showcb=False, showit=False)

    ani = animation.FuncAnimation(fig, update, np.arange(1, len(prb.times)), init_func=init, blit=False)

    
    if save_mp4: 
        ani.save(
            "currents1.mp4", writer="ffmpeg", fps=3, dpi=250, bitrate=0, 
            metadata={"title":"TDEM currents in a plate", "artist":"Lindsey Heagy"}
        )

In [ ]:
anihtml = ani.to_jshtml(fps=3)
HTML(anihtml)

In [ ]:
iSrc = 5
clim=[1e-17, 3e-14]


if make_movie: 

    fig, ax = plt.subplots(1, 1, figsize = (8, 8), dpi=290)

    out = plot_magnetic_flux(itime=1, iSrc=iSrc, clim=clim, ax=ax, showcb=True, showit=False)

    def init():
        [o.set_array(None) for o in out if isinstance(o, collections.QuadMesh)]
        return out

    def update(t):
        ax.patches = []
        ax.lines = []
        ax.clear()
        return plot_magnetic_flux(itime=t, iSrc=iSrc, clim=clim, ax=ax, showcb=False, showit=False)

    ani = animation.FuncAnimation(fig, update, np.arange(1, len(prb.times)), init_func=init, blit=False)
    
    if save_mp4: 
        ani.save(
            "magnetic_flux.mp4", writer="ffmpeg", fps=3, dpi=250, bitrate=0, 
            metadata={"title":"TDEM magnetic flux in a plate", "artist":"Lindsey Heagy"}
        )

In [ ]:
anihtml = ani.to_jshtml(fps=3)
HTML(anihtml)

## Presentation figures

In [ ]:
fig, axes = plt.subplots(3, 3, figsize = (22,6*3))
 
iSrc = 5
clim_j = [3e-13, 2e-9]
clim_b = [1e-17, 3e-14]

for i, itime in enumerate([1, 10, 22]):
    
    ax = axes[i, :]
    location  = srcList[iSrc].loc
    
    S = np.kron(np.ones(3), sigma)
    j = Utils.sdiag(S) * mesh.aveE2CCV * fields[srcList[iSrc], 'e', itime]
    
    b = mesh.aveF2CCV * fields[survey.srcList[iSrc], 'b', itime]

    dat1 = mesh.plotSlice(
        j, normal='Z', ind=int(indz), vType='CCv', view='vec', ax=ax[0], 
        range_x=xlim, range_y=ylim, 
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim_j, stream_threshold = 1e-12 if clim is not None else None
    )
    dat2 = mesh.plotSlice(
        j, normal='X', ind=int(indx), vType='CCv', view='vec', ax=ax[1], 
        range_x=xlim, range_y=zlim,
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'}, 
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim_j, stream_threshold = 1e-12 if clim is not None else None
    )
    
    dat3 = mesh.plotSlice(
        b, normal='Y', ind=int(indy), vType='CCv', view='vec', ax=ax[2],
        range_x=xlim, range_y=xlim,
        pcolorOpts={'norm':LogNorm(), 'cmap':'magma'},
        streamOpts={'arrowsize':2, 'color':'k'},
        clim=clim_b, stream_threshold = clim_b[0] if clim is not None else None
    )
    
    ax[0].plot(location[0], location[1], 'go', ms=20)
    ax[0].text(-600, 550, "Time: {:1.2f} ms".format(prb.times[itime]*1e3), color='w', fontsize=20)
    cb = plt.colorbar(dat1[0], orientation="horizontal", ax = ax[1])
    cb.set_label("Current density (A/m$^2$)")
    
    cb2 = plt.colorbar(dat3[0], ax=ax[2])
    cb2.set_label("Magnetic flux density (T)")
    ax[0].set_aspect(1)
    ax[1].set_aspect(1)
    ax[2].set_aspect(1)
    
    
    if itime == 1: 
        ax[0].set_title("Current density, Z={:1.0f}m".format(mesh.vectorCCz[indz]))
        ax[1].set_title("Current density, X={:1.0f}m".format(mesh.vectorCCx[indx]))
        ax[2].set_title("Magnetic flux, Y=0m")
    else:
        ax[0].set_title("")
        ax[1].set_title("")
        ax[2].set_title("")

plt.tight_layout()
plt.show()